In [1]:
from mxnet.gluon import nn 
from mxnet import nd 
net=nn.Sequential() 

In [2]:
with net.name_scope():
    # 第一层卷积 
    net.add(nn.Conv2D(channels=20,kernel_size=5))
    ### 添加了批量归一化层 
    net.add(nn.BatchNorm(axis=1)) 
    net.add(nn.Activation(activation='relu'))
    net.add(nn.MaxPool2D(pool_size=2,strides=2)) 
    # 第二层卷积  
    net.add(nn.Conv2D(channels=50,kernel_size=3)) 
    ### 添加了批量归一化层
    net.add(nn.BatchNorm(axis=1)) 
    net.add(nn.Activation(activation='relu'))
    net.add(nn.MaxPool2D(pool_size=2,strides=2)) 
    net.add(nn.Flatten()) 
    # 第一层全连接 
    net.add(nn.Dense(128,activation='relu')) 
    # 第二层全连接 
    net.add(nn.Dense(10)) 
    

In [14]:
import sys 
sys.path.append("..")
from mxnet import gluon 
from mxnet import autograd 
import time 
import utils 
import mxnet as mx
ctx=utils.try_gpu() 
# ctx=mx.cpu()
net.initialize(ctx=ctx) 
batch_size=256
train_data,test_data=utils.load_data_fashion_mnist(batch_size=batch_size)
softmax_cross_entropy=gluon.loss.SoftmaxCrossEntropyLoss() 
trainer=gluon.Trainer(net.collect_params(),'sgd',{'learning_rate':0.001})
start=time.time()
for epoch in range(5):
    train_loss=0.
    train_acc=0. 
    for data,label in train_data:
        label=label.as_in_context(ctx) 
        with autograd.record():
            output=net(data.as_in_context(ctx))  
            loss=softmax_cross_entropy(output,label) 
        loss.backward() 
        trainer.step(batch_size) 
        train_loss+=nd.mean(loss).asscalar() 
        train_acc+=utils.accuracy(output,label)
    test_acc=utils.evaluate_accuracy(test_data,net,ctx)
    print("Epoch %d . loss :%f ,train acc %f,Test acc%f "% 
          (epoch,train_loss/len(train_data),train_acc/len(train_data),test_acc)
          ) 
end=time.time() 
print("Time is : %f"%(end-start)) 

aaaaaaaa
Epoch 0 . loss :0.138548 ,train acc 0.951823,Test acc0.918770 
Epoch 1 . loss :0.138715 ,train acc 0.951656,Test acc0.918970 
Epoch 2 . loss :0.138454 ,train acc 0.951372,Test acc0.919171 
Epoch 3 . loss :0.138615 ,train acc 0.951689,Test acc0.918470 
Epoch 4 . loss :0.138510 ,train acc 0.951706,Test acc0.919171 
Time is : 8.105082
